<a href="https://colab.research.google.com/github/AshwinPradeep01/Fashion-reviews-using-BERT-pytorch/blob/main/FashionReviewsUsingPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Checking the GPU |Runtime and Installing packages**

In [ ]:
!nvidia-smi

Fri Nov 10 13:37:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
!pip install transformers tokenizers -q requests beautifulsoup4 pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.4 MB/s eta 0:00:00


### **Importing Necessary libraries**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

###**Importing the dataset**

In [ ]:
df = pd.read_csv('/content/Womens Clothing E-Commerce Reviews.csv')
df.rename(columns={"Unnamed: 0": "id","Review Text":"text"}, inplace=True)
df.set_index('id', inplace=True)
df.head()

,Clothing ID,Age,Title,text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
id,,,,,,,,,,
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


### **Preprocessing the Data**

In [ ]:
df = df[['text','Rating']]

In [ ]:
df = df.iloc[:8000]

In [ ]:
df.Rating.unique()

array([3, 4, 2, 1, 0])

In [ ]:
df.Rating.value_counts()

4    4444
3    1768
2     986
1     528
0     274
Name: Rating, dtype: int64

In [ ]:
labels = sorted(df.Rating.unique())

In [ ]:
label_dict = {}
for index, label in enumerate(labels):
  label_dict[label] = index

In [ ]:
label_dict

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4}

In [ ]:
# def sentscoremap(rating):
#   new_ratings = int(rating)
#   if new_ratings==1:
#     return 0
#   elif new_ratings==2:
#     return 1
#   elif new_ratings==3:
#     return 2
#   elif new_ratings==4:
#     return 3
#   elif new_ratings==5:
#     return 4

# df['Rating'] = df.Rating.apply(sentscoremap)
# df.head()

In [ ]:
df.Rating = df['Rating'].map(label_dict)

In [ ]:
df.head()

,text,Rating,data_type
id,,,
0,Absolutely wonderful - silky and sexy and comf...,3,train
1,Love this dress! it's sooo pretty. i happene...,4,train
2,I had such high hopes for this dress and reall...,2,train
3,"I love, love, love this jumpsuit. it's fun, fl...",4,train
4,This shirt is very flattering to all due to th...,4,val


## **Splitting the Dataset**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.Rating.values,
                                                  test_size=0.20,
                                                  random_state=42,
                                                  stratify=df.Rating.values)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

In [ ]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['Rating', 'data_type']).count()

text
Rating data_type      
0      train       219
       val          55
1      train       422
       val         106
2      train       789
       val         197
3      train      1415
       val         353
4      train      3555
       val         889

In [ ]:
df.text.isnull().sum()

0

In [ ]:
df.dropna(inplace =True)

## **Using Bert Tokenizer**

In [ ]:
import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

ModuleNotFoundError: ignored

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

NameError: ignored

In [ ]:
EPOCHS = 10
LEARNING_RATE = 1e-05

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].Rating.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].Rating.values)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
dataset_train = TensorDataset(input_ids_train,
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val,
                            attention_masks_val,
                           labels_val)

In [ ]:
len(dataset_train)

6400

In [ ]:
dataset_val.tensors

(tensor([[ 101, 1045, 3641,  ...,    0,    0,    0],
         [ 101, 2062, 1998,  ...,    0,    0,    0],
         [ 101, 2023, 2003,  ...,    0,    0,    0],
         ...,
         [ 101, 4121, 2007,  ...,    0,    0,    0],
         [ 101, 1045, 3641,  ...,    0,    0,    0],
         [ 101, 1045, 3866,  ...,    0,    0,    0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([3, 4, 2,  ..., 0, 3, 2]))

In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels = len(label_dict),
                                                      output_attentions = False,output_hidden_states = False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

## **Initializing the optimizer (ADAMW)**

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = torch.optim.AdamW(params = model.parameters(), lr = LEARNING_RATE,eps = 1e-8)

In [ ]:
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0,
                                            num_training_steps=len(dataloader_train)*EPOCHS)

In [ ]:
from sklearn.metrics import f1_score

def score(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return f1_score(labels_flat, preds_flat, average = 'weighted')

# **Accuracy per class**

In [ ]:
def accuracy(preds, labels):
  label_dict_inverse = {v: k for k, v in label_dict.items()}

  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  for label in np.unique(labels_flat):
    y_preds = preds_flat[labels_flat==label]
    y_true = labels_flat[labels_flat==label]
    print(f'Class: {label_dict_inverse[label]}')
    print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

## Training module

In [ ]:
import random

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in tqdm(dataloader_val):

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## **Training Loop**

Here the trainings epochs along with their training loss and validation loss have been illustrated.

In [ ]:
for epoch in tqdm(range(1, EPOCHS+1)):
    model.train()
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train,
                        desc='Epoch {:1d}'.format(epoch),
                        leave=False,
                        disable=False)

    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }

        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    #torch.save(model.state_dict(), f'Models/BERT_ft_Epoch{epoch}.model')

    tqdm.write('\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = score(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/1600 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.7283744747296441


  0%|          | 0/50 [00:00<?, ?it/s]

Validation loss: 0.7090627467632293
F1 Score (weighted): 0.7042779261785219


...

Validation loss: 2.032344399690628
F1 Score (weighted): 0.7032419579513788


Epoch 10:   0%|          | 0/1600 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.06975537569898733


  0%|          | 0/50 [00:00<?, ?it/s]

Validation loss: 2.0442014396190644
F1 Score (weighted): 0.7123385022174865


In [ ]:
torch.save(model,'/content/sample_data/sentiment_model.pth')
tokenizer.save_vocabulary('/content/sentiment_vocab.pth')

('/content/sentiment_vocab.pth',)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Datasets

Mounted at /content/drive
/content/drive/MyDrive/Datasets


In [ ]:
demotext = "Absolutely wonderful - silky and sexy and comfortable"
tokens =tokenizer.encode(demotext)

# **Accuracy per class**

In [ ]:
accuracy(predictions, true_vals)

Class: 0
Accuracy:21/55

Class: 1
Accuracy:49/106

Class: 2
Accuracy:106/197

Class: 3
Accuracy:182/353

Class: 4
Accuracy:785/889



## **Demonstration of the Sample**

In [ ]:
import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

In [ ]:
bertmodel = torch.load('/content/drive/MyDrive/Datasets/sentiment_model.pth',map_location=torch.device('cpu'))

In [ ]:
review_text = "she does not look nice"

def get_sentiment(review_text):
  encoded_review = tokenizer.encode_plus(review_text, max_length=512,
                                        add_special_tokens=True, return_token_type_ids = False,
                                        pad_to_max_length=True,
                                        return_attention_mask=True,
                                        truncation=True,
                                        return_tensors='pt')

  input_ids = encoded_review['input_ids']

  attention_mask = encoded_review['attention_mask']

  output = bertmodel(input_ids,attention_mask)

  prediction = int(torch.argmax(output.logits))

  pred_dict ={
      0:"sad",
      1:"sad",
      2:"neutral",
      3:"happy",
      4:"happy"
  }

  prediction = pred_dict[prediction]
  print(f"Review text:{review_text}")
  print(f"Sentiment:{prediction}")

get_sentiment(review_text)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Review text:she does not look nice
Sentiment:sad


In [ ]:
sample_review = "Absolutely wonderful - silky and sexy and comfortable"
get_sentiment(sample_review)

Review text:Absolutely wonderful - silky and sexy and comfortable
Sentiment:happy
